# Chroma

In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [6]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [7]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f32c4291-99da-43a4-ac26-614102edeac2',
 'f864256b-c9e9-427b-a40a-0775a0ae3781',
 'f104959c-ce41-418d-bd78-92d5c8af0525',
 '5b263e82-15ed-4077-bee7-980c15fb7d73',
 '6d47519a-a04f-41de-8f55-f4992711af2e',
 '34ec8977-d5fc-4b68-bccf-1dba8a9e8ecd',
 'eee7871f-f272-4865-87fe-2dceb4360591',
 '4c559465-72a1-48f4-9728-9ff8cec05318',
 '5f36587b-86da-495b-8141-6d44aacee366',
 '7ea5e6c8-42d2-4b8a-b76a-24914fe9bb53']

In [8]:
updated_document_1 = Document(
    page_content="I had chocolate chip pancakes and fried eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

updated_document_2 = Document(
    page_content="The weather forecast for tomorrow is sunny and warm, with a high of 82 degrees.",
    metadata={"source": "news"},
    id=2,
)

vector_store.update_document(document_id=uuids[0], document=updated_document_1)
# You can also update multiple documents at once
vector_store.update_documents(
    ids=uuids[:2], documents=[updated_document_1, updated_document_2]
)

In [9]:
vector_store.delete(ids=uuids[-1])

## Query vector store

In [10]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [11]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.807169] The weather forecast for tomorrow is sunny and warm, with a high of 82 degrees. [{'source': 'news'}]


In [12]:
results = vector_store.similarity_search_by_vector(
    embedding=embeddings.embed_query("I love green eggs and ham!"), k=1
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* I had chocolate chip pancakes and fried eggs for breakfast this morning. [{'source': 'tweet'}]


In [13]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5}
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='5b263e82-15ed-4077-bee7-980c15fb7d73', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

## Test

In [14]:
vector_store = Chroma(
    collection_name="test_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [16]:
document_1 = Document(
    page_content="This is a test document.",
    metadata={"source": "test"},
    id=1,
)

document_2 = Document(
    page_content="I love chocolate.",
    metadata={"source": "food"},
    id=2,
)

document_3 = Document(
    page_content="I love green eggs and ham.",
    metadata={"source": "food"},
    id=3,
)

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents)

['1', '2', '3']

In [18]:
vector_store.delete(ids=['3'])

In [19]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [20]:
document_3 = Document(
    page_content="I love green eggs and ham.",
    metadata={"source": "food"},
    id=90,
)

vector_store.add_documents(documents=[document_3], ids=[str(uuid4())])

['7250848f-716f-4f87-ab17-2d4610d39cd1']